# PyQuantumForecaster Demo: Sine Wave Prediction

This notebook demonstrates using the QuantumForecaster to predict oscillatory time series data.

In [ ]:
import sys
sys.path.append('../src')

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from pyquantumforecaster.core import QuantumForecaster

In [ ]:
# Generate synthetic sine wave data
def generate_sine_data(n_points=1000, freq=0.1, noise=0.1):
    t = np.linspace(0, 100, n_points)
    data = np.sin(2 * np.pi * freq * t) + noise * np.random.randn(n_points)
    return data.astype(np.float32)

# Create dataset
data = generate_sine_data()
print(f"Generated {len(data)} data points")
print(f"Data range: [{data.min():.3f}, {data.max():.3f}]")

In [ ]:
# Prepare training data with sliding window
window_size = 50
X, y = [], []

for i in range(len(data) - window_size):
    X.append(data[i:i+window_size])
    y.append(data[i+window_size])

X = torch.FloatTensor(X)
y = torch.FloatTensor(y).unsqueeze(1)

# Split data
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Initialize the quantum forecaster
model = QuantumForecaster(window_size=window_size, n_qubits=8, n_layers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Quantum weights shape: {model.q_weights.shape}")

In [ ]:
# Training loop
epochs = 100
batch_size = 32
losses = []

model.train()
for epoch in range(epochs):
    epoch_loss = 0
    
    # Mini-batch training
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / (len(X_train) // batch_size)
    losses.append(avg_loss)
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {avg_loss:.6f}")

print(f"Training completed. Final loss: {losses[-1]:.6f}")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
# Evaluate on test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test)
    test_loss = criterion(test_predictions, y_test)
    
print(f"Test MSE: {test_loss.item():.6f}")
print(f"Test RMSE: {np.sqrt(test_loss.item()):.6f}")

In [ ]:
# Visualize predictions vs actual
plt.figure(figsize=(12, 6))

# Plot actual vs predicted for a subset of test data
n_plot = 200
actual = y_test[:n_plot].numpy().flatten()
predicted = test_predictions[:n_plot].numpy().flatten()

plt.plot(actual, label='Actual', alpha=0.7)
plt.plot(predicted, label='Predicted', alpha=0.7)
plt.title('Quantum Forecaster: Actual vs Predicted')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate prediction accuracy metrics
from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(actual, predicted)
mae = mean_absolute_error(actual, predicted)

print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Absolute Percentage Error: {np.mean(np.abs((actual - predicted) / actual)) * 100:.2f}%")